In [ ]:
## GEOG458
## Xinbo Lu, Wenjin Yu, Jody Tran
## 2/15/2019
## Group Lab 3


In [3]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib as mpl
%matplotlib inline

In [2]:
WA_bg = gpd.read_file("C:/Users/LXB19/Documents/GEOG458/GroupLab3/saep_bg10_Copy/saep_bg10.shp")
FIPS_to_CountyNames_gdf = gpd.read_file("C:/Users/LXB19/Documents/GEOG458/GroupLab3/WashingtonFIPS_copy.dbf")
WA_bg.head(5)

,STATEFP10,COUNTYFP10,TRACTCE10,BLKGRPCE10,GEOID10,NAMELSAD10,MTFCC10,FUNCSTAT10,INTPTLON10,INTPTLAT10,...,OHU2014,OHU2015,OHU2016,OHU2017,OHU2018,COHU00_10,PCOHU00_10,COHU10_18,PCOHU10_18,geometry
0,53,001,950100,1,530019501001,Block Group 1,G5030,S,-118.398815,47.150809,...,277.494,279.680,287.530,287.522,287.758,33.751,0.143,17.758,0.066,"POLYGON ((2077217.074095237 640954.0110817049,..."
1,53,001,950100,2,530019501002,Block Group 2,G5030,S,-118.351143,47.135101,...,524.831,521.228,517.427,516.077,524.645,3.398,0.007,-0.355,0.000,"POLYGON ((2165913.051240579 657352.8005365322,..."
2,53,001,950100,3,530019501003,Block Group 3,G5030,S,-118.382148,47.138222,...,246.344,243.965,242.033,241.365,245.491,-26.312,-0.097,-0.509,-0.002,"POLYGON ((2166254.454566129 657810.7137267586,..."
3,53,001,950200,1,530019502001,Block Group 1,G5030,S,-118.219484,46.874027,...,208.923,203.761,205.502,204.547,202.854,-21.255,-0.093,-5.146,-0.025,"POLYGON ((2150159.78406948 554295.8999707697, ..."
4,53,001,950200,2,530019502002,Block Group 2,G5030,S,-118.522323,46.968569,...,285.857,286.251,278.398,278.400,267.983,-1.330,-0.005,-22.017,-0.076,"POLYGON ((2082999.834788324 575018.8611698836,..."


In [3]:
print "Initial table: "
print FIPS_to_CountyNames_gdf.head(5)

Initial table: 
  CountyName FIPSCounty geometry
0      Adams        001     None
1     Asotin        003     None
2     Benton        005     None
3     Chelan        007     None
4    Clallam        009     None


In [4]:
FIPS_to_CountyNames_gdf.rename(columns={'FIPSCounty':'COUNTYFP10'},inplace=True)
FIPS_to_CountyNames_gdf.drop('geometry', axis=1, inplace=True)
print
print "After preparation for merge (join): "
print FIPS_to_CountyNames_gdf.head()


After preparation for merge (join): 
  CountyName COUNTYFP10
0      Adams        001
1     Asotin        003
2     Benton        005
3     Chelan        007
4    Clallam        009


In [5]:
JOINING_KEY = 'COUNTYFP10'
WA_bg_with_CountyName = WA_bg.merge(FIPS_to_CountyNames_gdf, on=JOINING_KEY)
WA_bg_with_CountyName.head()

,STATEFP10,COUNTYFP10,TRACTCE10,BLKGRPCE10,GEOID10,NAMELSAD10,MTFCC10,FUNCSTAT10,INTPTLON10,INTPTLAT10,...,OHU2015,OHU2016,OHU2017,OHU2018,COHU00_10,PCOHU00_10,COHU10_18,PCOHU10_18,geometry,CountyName
0,53,001,950100,1,530019501001,Block Group 1,G5030,S,-118.398815,47.150809,...,279.680,287.530,287.522,287.758,33.751,0.143,17.758,0.066,"POLYGON ((2077217.074095237 640954.0110817049,...",Adams
1,53,001,950100,2,530019501002,Block Group 2,G5030,S,-118.351143,47.135101,...,521.228,517.427,516.077,524.645,3.398,0.007,-0.355,0.000,"POLYGON ((2165913.051240579 657352.8005365322,...",Adams
2,53,001,950100,3,530019501003,Block Group 3,G5030,S,-118.382148,47.138222,...,243.965,242.033,241.365,245.491,-26.312,-0.097,-0.509,-0.002,"POLYGON ((2166254.454566129 657810.7137267586,...",Adams
3,53,001,950200,1,530019502001,Block Group 1,G5030,S,-118.219484,46.874027,...,203.761,205.502,204.547,202.854,-21.255,-0.093,-5.146,-0.025,"POLYGON ((2150159.78406948 554295.8999707697, ...",Adams
4,53,001,950200,2,530019502002,Block Group 2,G5030,S,-118.522323,46.968569,...,286.251,278.398,278.400,267.983,-1.330,-0.005,-22.017,-0.076,"POLYGON ((2082999.834788324 575018.8611698836,...",Adams


In [ ]:
WA_bg_with_CountyName_wgs84 = WA_bg_with_CountyName.to_crs(epsg=4326)
import os  
for county in FIPS_to_CountyNames_gdf['CountyName']:
    filename_to_output = county+".geojson"
    print "Outputting "+filename_to_output
    try: 
        os.remove(filename_to_output)
    except OSError:
        pass
    (WA_bg_with_CountyName_wgs84[WA_bg_with_CountyName['CountyName'] == county]
     .to_file(filename_to_output,driver="GeoJSON"))

In [6]:
## Part 2.2
## Print out a ranked list, in descending order, of the ten largest total populations 
## in 2017 of counties in Washington, according to this dataset.

pop2018_series = WA_bg_with_CountyName.groupby(['CountyName']).sum()['POP2018'].rank(ascending=False).head(10)
top10_POP2017_county = pop2018_series.index.tolist()

In [4]:
## Part 3

## Read the Washington State shp
WA_shp = gpd.read_file("C:/Users/LXB19/Documents/GEOG458/GroupLab3/saep_bg10_Copy/saep_bg10.shp")


## Calculate Population density in 2018
WA_shp['Pop_Per_sqML_2018'] = WA_shp.POP2018/WA_shp.ALANDMI + 10**-8


In [5]:
## Part 3.1
## Use population density equal or greater than 500 as the urban area. Otherwise, rural.

WA_shp['Urban_2018'] = np.nan
WA_shp.loc[WA_shp.Pop_Per_sqML_2018 >= 500, 'Urban_2018'] = 'urban'
WA_shp.loc[WA_shp.Pop_Per_sqML_2018 < 500, 'Urban_2018'] = 'rural'

In [6]:
## Part 3.2
## Calculates and prints out to the screen what percentage of the population of the state 
##    is urbanized in the most recent year.

UrbanArea_2018 = WA_shp[(WA_shp['Pop_Per_sqML_2018'] >= 500)]
UrbanPopPercent_2018 = round(sum(UrbanArea_2018.POP2018 / sum(WA_shp.POP2018)) * 100, 2)
print "The percentage of the population of the state is urbanized in 2018 is", UrbanPopPercent_2018, "%"

The percentage of the population of the state is urbanized in 2018 is 80.48 %


In [6]:
UrbanArea_2018

,STATEFP10,COUNTYFP10,TRACTCE10,BLKGRPCE10,GEOID10,NAMELSAD10,MTFCC10,FUNCSTAT10,INTPTLON10,INTPTLAT10,...,OHU2016,OHU2017,OHU2018,COHU00_10,PCOHU00_10,COHU10_18,PCOHU10_18,geometry,Pop_Per_sqML_2018,Urban_2018
10,53,001,950300,5,530019503005,Block Group 5,G5030,S,-119.199591,46.808016,...,327.259,334.314,339.604,13.642,0.045,21.604,0.068,"POLYGON ((1969258.198712114 547133.5683372517,...",538.691147,urban
11,53,001,950400,1,530019504001,Block Group 1,G5030,S,-119.159615,46.828822,...,416.060,423.179,425.375,7.551,0.019,29.375,0.074,"POLYGON ((1977239.155600399 546710.0576105967,...",4049.804954,urban
12,53,001,950400,2,530019504002,Block Group 2,G5030,S,-119.168580,46.833542,...,535.584,536.836,536.612,60.509,0.142,50.612,0.104,"POLYGON ((1977183.411740516 551297.6280722701,...",1847.043522,urban
13,53,001,950500,1,530019505001,Block Group 1,G5030,S,-119.152869,46.810221,...,653.936,690.947,701.545,182.463,0.401,63.545,0.100,"POLYGON ((1982308.672673395 547391.738878502, ...",698.070104,urban
14,53,001,950500,2,530019505002,Block Group 2,G5030,S,-119.178195,46.811467,...,712.737,772.728,785.062,71.316,0.121,126.062,0.191,"POLYGON ((1971831.33646277 537762.5527730057, ...",2232.074436,urban
18,53,003,960200,1,530039602001,Block Group 1,G5030,S,-117.069978,46.379782,...,458.259,460.931,462.794,34.900,0.083,8.794,0.019,"POLYGON ((2510271.06743315 398760.139835091, 2...",775.555556,urban
19,53,003,960200,2,530039602002,Block Group 2,G5030,S,-117.080996,46.384562,...,506.675,507.283,510.245,15.885,0.033,8.245,0.016,"POLYGON ((2501356.630242492 399864.0424212132,...",2829.744186,urban
20,53,003,960200,3,530039602003,Block Group 3,G5030,S,-117.094021,46.392495,...,348.156,353.423,360.857,61.939,0.236,36.857,0.114,"POLYGON ((2497815.046612344 407182.670825434, ...",773.049530,urban
22,53,003,960300,1,530039603001,Block Group 1,G5030,S,-117.055951,46.422832,...,502.011,500.975,503.316,13.883,0.029,4.316,0.009,"POLYGON ((2505416.177773451 414724.5622559617,...",1421.166244,urban
23,53,003,960300,2,530039603002,Block Group 2,G5030,S,-117.054958,46.414740,...,424.987,426.157,428.849,14.902,0.036,1.849,0.004,"POLYGON ((2508164.018435895 412150.5568534232,...",5334.540698,urban


In [7]:
## Part 3.4
##Write code that calculates and prints out to the screen what percentage 
##     of the land area of the state is urbanized in the most recent year.

## Calculating Urban area size
urbanArea_size = UrbanArea_2018.ALANDMI.sum()

## calculating the total area size
totalArea = WA_shp.ALANDMI.sum()

## getting the percentage
UrbanAreaPercent_2018 = round(urbanArea_size/totalArea * 100,2)

## Print out the statement
print "The percentage of the urbanized area size of the state in 2018 is", UrbanAreaPercent_2018, "%"

The percentage of the urbanized area size of the state in 2018 is 3.47 %


In [27]:
## Add a column to the shapefile's data table for whether block groups have 'urbanized', 
## 'no change in category', or 'deurbanized' over the previous decade (i.e. if most recent year is ’18, 
## then ’08-’18), using the above classification for urban/rural you decided on. 
## Represent it as a categorical change (a string of text), not in terms of some numerical change. 
## Fill in the field for all rows! 

## Population density 2008
WA_shp['Pop_Per_sqML_2008'] = WA_shp.POP2008/WA_shp.ALANDMI + 10**-8

## Creating new column for comparison
## Urban_2008 shows whether the block is urban or rural
WA_shp['Urban_2008'] = np.nan

## Create the column to show the decade change (comparing 2008 vs 2018)
WA_shp['Urban_2008_DecadeChange']= np.nan

In [28]:
## Set the urban/rural definition to each block
WA_shp.loc[WA_shp.Pop_Per_sqML_2008 >= 500, 'Urban_2008'] = 'urban'
WA_shp.loc[WA_shp.Pop_Per_sqML_2008 < 500, 'Urban_2008'] = 'rural'

In [30]:
for i in range(len(WA_shp)):
    ## case 1: if 2018 is urban, and 2008 is rural, then urbanized"
    if (WA_shp.Urban_2018.loc[i] == 'urban' and WA_shp.Urban_2008[i] =='rural'):
        WA_shp.Urban_2008_DecadeChange.loc[i] = 'urbanized'
    ## case 2: if 2018 is rural, and 2008 is urban, then deurbanized"
    elif(WA_shp.Urban_2018.loc[i] == 'rural' and WA_shp.Urban_2008[i] == 'urban'):
        WA_shp.Urban_2008_DecadeChange.loc[i] = 'deurbanized'
    ## case 3, 2008 and 2018 are both rural/urbanized, then no change in category"
    else:
        WA_shp.Urban_2008_DecadeChange.loc[i] = 'no change in category'

In [31]:
WA_shp.head()

,STATEFP10,COUNTYFP10,TRACTCE10,BLKGRPCE10,GEOID10,NAMELSAD10,MTFCC10,FUNCSTAT10,INTPTLON10,INTPTLAT10,...,COHU00_10,PCOHU00_10,COHU10_18,PCOHU10_18,geometry,Pop_Per_sqML_2018,Urban_2018,Pop_Per_sqML_2008,Urban_2008,Urban_2008_DecadeChange
0,53,001,950100,1,530019501001,Block Group 1,G5030,S,-118.398815,47.150809,...,33.751,0.143,17.758,0.066,"POLYGON ((2077217.074095237 640954.0110817049,...",1.270921,rural,1.192891,rural,no change in category
1,53,001,950100,2,530019501002,Block Group 2,G5030,S,-118.351143,47.135101,...,3.398,0.007,-0.355,0.000,"POLYGON ((2165913.051240579 657352.8005365322,...",303.724695,rural,308.274775,rural,no change in category
2,53,001,950100,3,530019501003,Block Group 3,G5030,S,-118.382148,47.138222,...,-26.312,-0.097,-0.509,-0.002,"POLYGON ((2166254.454566129 657810.7137267586,...",171.655748,rural,173.292027,rural,no change in category
3,53,001,950200,1,530019502001,Block Group 1,G5030,S,-118.219484,46.874027,...,-21.255,-0.093,-5.146,-0.025,"POLYGON ((2150159.78406948 554295.8999707697, ...",1.006330,rural,1.087854,rural,no change in category
4,53,001,950200,2,530019502002,Block Group 2,G5030,S,-118.522323,46.968569,...,-1.330,-0.005,-22.017,-0.076,"POLYGON ((2082999.834788324 575018.8611698836,...",5.199997,rural,5.331252,rural,no change in category


In [42]:
## calculates and prints out how many block groups urbanized and how many deurbanized over the previous decade.

## calculate how many block groups are urbanized by using the "urbanized" attribute
UrbanizedLastDecade = len(WA_shp[WA_shp.Urban_2008_DecadeChange =='urbanized'].index)

## calculate how many block group are deurbanized by using the "deurbanized" attribute
DeurbanizedLastDecade = len(WA_shp[WA_shp.Urban_2008_DecadeChange =='deurbanized'].index)

## print out the statement
print UrbanizedLastDecade,"blocks are urbanized over the previous decade, and",\
        DeurbanizedLastDecade, "blocks are deurbanized over the previous decade"

65 blocks are urbanized over the previous decade, and 0 blocks are deurbanized over the previous decade
